# OpenAI API
Če imaš težave z računanjem vektorjev; Funkcije za vektorizacijo z OpenAI API.   
Spet kličem vektorizacijo na stolpcu 'Text' in dobim stolpec z vektorjem 'Embedding'

In [1]:
!pip install pandas numpy tiktoken openai networkx numpy scikit-learn

In [2]:
import os
import pandas as pd

excel_path = os.path.join('..', 'data', 'Reworkan data.xlsx')  # Za primer sem dal krajši file
data = pd.read_excel(excel_path)
data.head()

,Article ID,Datum,Dan,Teden,Article created,Tip medija,Podtip medija,Media uuid,Medij,Rubric uuid,...,Pregled objav - citiranost virov,Razredi Indeksa medijske podobe,Doseg,Naklonjenost teme,Osrednja tema,Ključna sporočila,Fotografija,Omemba v naslovu,Omemba v podnaslovu,Tekst
0,0c801540-d013-11ed-9f09-af460eec62b1,2023-04-01,Saturday,13,2023-04-01 00:26:15.103,tisk,tisk,7eadd15d-28aa-485c-8c00-1e11cff87069,Delo - Sobotna priloga,97dd7328-45de-43f9-8732-d0b8f105f27c,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),zelo dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Jerneja Terčon Strokovnjakinja za nevrorazličn...
1,2945adc3-d01b-11ed-83eb-0bd28cf4c8f0,2023-04-01,Saturday,13,2023-04-01 01:24:19.348,tisk,tisk,7eadd15d-28aa-485c-8c00-1e11cff87069,Delo - Sobotna priloga,97dd7328-45de-43f9-8732-d0b8f105f27c,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),zelo dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"Ni več upanja, povozil gaje tank / SAMO ZVER m..."
2,3e1f0d89-d010-11ed-83eb-0bd28cf4c8f0,2023-04-01,Saturday,13,2023-04-01 00:06:09.863,tisk,tisk,cf08eb8c-1e50-4884-ae0a-9ccf7d735817,Slovenske novice,e2d7d641-5fce-428c-b204-1c4b8f5a8eaa,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Z rutinske operaciji e je vrnil na vozičku Grb...
3,43edd0d8-d00b-11ed-9d48-dfb487f9037c,2023-04-01,Saturday,13,2023-03-31 23:30:32.123,tisk,tisk,8d0af10b-3831-4342-9add-b2232332ae02,Večer - V soboto,c3f5a2d2-e46f-4888-8f04-afa6427e1818,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"ZDRAVJE VRTILJAK IDEJ, ANALIZ IN INTERESOV ZDR..."
4,8cd51594-d01c-11ed-9f09-af460eec62b1,2023-04-01,Saturday,13,2023-04-01 01:34:15.879,tisk,tisk,2fd717ed-78ba-4f63-b257-cd096acb6bda,Delo,34846d6e-c759-4896-b1c8-4c85f69762d7,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Protest Od jutri do preklica zdravniška stavka...


Pazi na okoljske spremenljivke:

In [3]:
print(f'OPENAI_API_KEY is set: {len(os.getenv('OPENAI_API_KEY'))}')
print(f'OPENAI_ORG_ID is set: {len(os.getenv('OPENAI_ORG_ID'))}')

OPENAI_API_KEY is set: 72
OPENAI_ORG_ID is set: 28


Definiram OpenAI API funkcije za vektorizacijo:

In [4]:
import tiktoken
import openai

def openai_embed(df: pd.DataFrame, col_name: str, target_col_name: str = 'Embedding'):
    def _oai_embed(text, encoding):
        tokens = encoding.encode(text)[:8191]
        embedding = openai.embeddings.create(  # call OpenAI
            input=tokens, model="text-embedding-ada-002"
        )
        return embedding.data[0].embedding
    
    encoding = tiktoken.get_encoding('cl100k_base')
    df[target_col_name] = df[col_name].apply(_oai_embed, encoding=encoding)
    return data

Kličem OpenAI API, da dobim embeddinge (vsak članek posebej zato traja):

In [5]:
data = openai_embed(data, 'Tekst', 'Embedding')
data[['Article ID', 'Datum', 'Medij', 'Tekst', 'Embedding']].head(100)

,Article ID,Datum,Medij,Tekst,Embedding
0,0c801540-d013-11ed-9f09-af460eec62b1,2023-04-01 00:00:00,Delo - Sobotna priloga,Jerneja Terčon Strokovnjakinja za nevrorazličn...,"[-0.0275590643286705, 0.01836414821445942, 0.0..."
1,2945adc3-d01b-11ed-83eb-0bd28cf4c8f0,2023-04-01 00:00:00,Delo - Sobotna priloga,"Ni več upanja, povozil gaje tank / SAMO ZVER m...","[-0.0037107784301042557, -0.005234049633145332..."
2,3e1f0d89-d010-11ed-83eb-0bd28cf4c8f0,2023-04-01 00:00:00,Slovenske novice,Z rutinske operaciji e je vrnil na vozičku Grb...,"[-0.01865057460963726, 0.011088660918176174, 0..."
3,43edd0d8-d00b-11ed-9d48-dfb487f9037c,2023-04-01 00:00:00,Večer - V soboto,"ZDRAVJE VRTILJAK IDEJ, ANALIZ IN INTERESOV ZDR...","[-0.0061466507613658905, 0.020326800644397736,..."
4,8cd51594-d01c-11ed-9f09-af460eec62b1,2023-04-01 00:00:00,Delo,Protest Od jutri do preklica zdravniška stavka...,"[-0.0018400881672278047, 0.011532104574143887,..."
...,...,...,...,...,...
94,77e71000-d4f9-11ed-83eb-0bd28cf4c8f0,2023-04-07 06:03:00,Reporter.si,Slovenija Silvester Šurla 07. apr. 2023 Deli n...,"[-0.005664257798343897, 0.016057895496487617, ..."
95,c1b38c44-d4fc-11ed-9d48-dfb487f9037c,2023-04-07 06:26:00,24ur.com,Električni skiroji lahko pripomorejo k večji m...,"[0.003973044920712709, -0.007546853274106979, ..."
96,b75ccef4-d50c-11ed-9246-2b5ebef623ad,2023-04-07 08:22:00,Sta.si,V Univerzitetnem kliničnem centru (UKC) Ljublj...,"[0.022891219705343246, 0.0066061485558748245, ..."
97,6f0e7f09-d50f-11ed-9f09-af460eec62b1,2023-04-07 08:40:00,24ur.com,V Univerzitetnem kliničnem centru Ljubljana so...,"[0.021619148552417755, 0.007398754823952913, 0..."


Funkcija za clusterizacijo

In [6]:
import networkx as nx
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
# Assuming your data variable is a DataFrame that you have already prepared
# Continuing from the last step of the previous script

def cluster_louvain(df: pd.DataFrame, col_name: str = 'Embedding', similarity_threshold: float = 0.96):
    print("Clustering ...")
    embeddings = np.array(df[col_name].to_list())
    labels = [0] * len(embeddings)
    x = cosine_similarity(embeddings, embeddings)
    similarity_matrix = x > similarity_threshold
    graph = nx.from_numpy_array(similarity_matrix)
    communities = nx.algorithms.community.louvain_communities(graph, resolution=0.1)
    for community in communities:
        initial_member = min(community)
        for member in community:
            labels[member] = initial_member

    df['Cluster'] = pd.Series(labels)
    print("Clustered")
    return df

Zračunam clustering

In [7]:
data = cluster_louvain(data, 'Embedding', 0.96)  # <- similarity_threshold
data = data.drop(columns=['Embedding'])  # remove the vectors from data we don't need them anymore
data[['Article ID', 'Datum', 'Medij', 'Cluster', 'Tekst']].head(100)

Clustering ...
Clustered


,Article ID,Datum,Medij,Cluster,Tekst
0,0c801540-d013-11ed-9f09-af460eec62b1,2023-04-01 00:00:00,Delo - Sobotna priloga,0,Jerneja Terčon Strokovnjakinja za nevrorazličn...
1,2945adc3-d01b-11ed-83eb-0bd28cf4c8f0,2023-04-01 00:00:00,Delo - Sobotna priloga,1,"Ni več upanja, povozil gaje tank / SAMO ZVER m..."
2,3e1f0d89-d010-11ed-83eb-0bd28cf4c8f0,2023-04-01 00:00:00,Slovenske novice,2,Z rutinske operaciji e je vrnil na vozičku Grb...
3,43edd0d8-d00b-11ed-9d48-dfb487f9037c,2023-04-01 00:00:00,Večer - V soboto,3,"ZDRAVJE VRTILJAK IDEJ, ANALIZ IN INTERESOV ZDR..."
4,8cd51594-d01c-11ed-9f09-af460eec62b1,2023-04-01 00:00:00,Delo,4,Protest Od jutri do preklica zdravniška stavka...
...,...,...,...,...,...
94,77e71000-d4f9-11ed-83eb-0bd28cf4c8f0,2023-04-07 06:03:00,Reporter.si,25,Slovenija Silvester Šurla 07. apr. 2023 Deli n...
95,c1b38c44-d4fc-11ed-9d48-dfb487f9037c,2023-04-07 06:26:00,24ur.com,95,Električni skiroji lahko pripomorejo k večji m...
96,b75ccef4-d50c-11ed-9246-2b5ebef623ad,2023-04-07 08:22:00,Sta.si,96,V Univerzitetnem kliničnem centru (UKC) Ljublj...
97,6f0e7f09-d50f-11ed-9f09-af460eec62b1,2023-04-07 08:40:00,24ur.com,96,V Univerzitetnem kliničnem centru Ljubljana so...
